# Train **YOLOv7** on URPC2019 v3 (Images)

This Colab trains a YOLOv7 detector using the URPC2019 v3 dataset from Roboflow (exported as **YOLOv7**).

## Steps
1. Install deps & clone YOLOv7
2. Authenticate to Roboflow and download dataset (YOLOv7 format)
3. Train YOLOv7 (default: yolov7-tiny)
4. Evaluate & save best weights
5. (Optional) Export to ONNX


In [ ]:
# --- PyTorch 2.5.1 (pre-2.6 default for torch.load) ---
!pip install --quiet --upgrade --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1

!pip -q install roboflow
%cd /content
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip -q install -r requirements.txt
print('YOLOv7 ready.')

In [ ]:
import os
from roboflow import Roboflow
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Roboflow settings (configure in .env or environment)
RF_API_KEY = os.getenv('ROBOFLOW_API_KEY')
if not RF_API_KEY:
    raise ValueError("Missing ROBOFLOW_API_KEY in .env or environment")

# Download dataset
rf = Roboflow(api_key=RF_API_KEY)
dataset = rf.workspace("underwater-fish-f6cri").project("urpc2019-nrbk1").version(3).download("yolov7")
print(f'Dataset downloaded to: {dataset.location}')

In [ ]:
import os, glob, shutil
import torch.serialization

# Enable numpy serialization for PyTorch 2.6+ compatibility
torch.serialization.add_safe_globals(['numpy._core.multiarray._reconstruct'])

%cd /content/yolov7

# Training settings (configure in .env or environment)
CFG = os.getenv('CFG', 'cfg/training/yolov7-tiny.yaml')
WEIGHTS = os.getenv('WEIGHTS', '')  # Empty for training from scratch
EPOCHS = int(os.getenv('EPOCHS', 50))
IMG_SIZE = int(os.getenv('IMG_SIZE', 640))
BATCH = int(os.getenv('BATCH', 16))

# Train YOLOv7
!python train.py --workers 8 --device 0 --batch-size {BATCH} --epochs {EPOCHS} \
    --img {IMG_SIZE} {IMG_SIZE} --data {dataset.location + '/data.yaml'} \
    --cfg {CFG} --weights '{WEIGHTS}' --name urpc2019_yolov7

# Save best weights
best_ckpts = sorted(glob.glob('runs/train/urpc2019_yolov7*/weights/best.pt'))
if not best_ckpts:
    raise SystemExit("Training failed - no best.pt found. Check errors above.")

dst_dir = '/content/urpc2019-fish-detector-yolov7/weights'
os.makedirs(dst_dir, exist_ok=True)
shutil.copy2(best_ckpts[-1], dst_dir + '/best.pt')
print(f'Saved best weights to {dst_dir}/best.pt')

/content/yolov7
2025-08-31 09:58:53.654528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756634333.674837   13099 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756634333.681182   13099 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756634333.697542   13099 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756634333.697572   13099 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756634333.697576   13099 computation_placer.cc:177] comput

In [21]:
import os
import glob
import random
import shutil
from google.colab import files

# Define the path to your test images
test_images_path = '/content/yolov7/URPC2019-3/test/images' # Assuming this is the path based on the previous cell output

# Get a list of all image files in the test directory
all_test_images = glob.glob(os.path.join(test_images_path, '*.jpg')) + \
                  glob.glob(os.path.join(test_images_path, '*.jpeg')) + \
                  glob.glob(os.path.join(test_images_path, '*.png'))

# Check if there are enough test images
if len(all_test_images) < 50:
    print(f"Warning: Only {len(all_test_images)} test images found. Zipping all of them.")
    images_to_zip = all_test_images
else:
    # Select 50 random test images
    images_to_zip = random.sample(all_test_images, 50)

# Create a temporary directory to store the selected images
zip_dir = '/content/selected_test_images'
os.makedirs(zip_dir, exist_ok=True)

# Copy the selected images to the temporary directory
for img_path in images_to_zip:
    shutil.copy(img_path, zip_dir)

# Zip the temporary directory
zip_filename = 'selected_test_images.zip'
shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', zip_dir)

print(f"Created {zip_filename}. You can download it using the files.download() function.")

# Optional: Download the zip file
# files.download(zip_filename)

Created selected_test_images.zip. You can download it using the files.download() function.


In [22]:
# You can use this cell to download the zip file after the previous cell finishes
from google.colab import files
files.download('selected_test_images.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>